In [10]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense
from flower import FederatedServer, FederatedClient
import flower.logger as logger

# Loading the pretrained model(GAN v2)
gan_model = load_model('trained_generated_model.h5', compile=False)

# Define the generator model
def define_generator(latent_dim, n_outputs=9):
    model = tf.keras.Sequential()
    model.add(Dense(15, activation='relu', kernel_initializer='he_uniform', input_dim=latent_dim))
    model.add(Dense(30, activation='relu'))
    model.add(Dense(n_outputs, activation='linear'))
    return model

# Define the discriminator model
def define_discriminator(n_inputs=9):
    model = tf.keras.Sequential()
    model.add(Dense(25, activation='relu', kernel_initializer='he_uniform', input_dim=n_inputs))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Define the GAN model
def define_gan(generator, discriminator):
    discriminator.trainable = False
    model = tf.keras.Sequential()
    model.add(generator)
    model.add(discriminator)
    model.compile(loss='binary_crossentropy', optimizer='adam')
    return model

# Define a virtual federated client
class GANFedAvgClient(FederatedClient):
    def __init__(self, model, gan_generator):
        super().__init__()
        self.model = model
        self.gan_generator = gan_generator
        self.local_data = None

    async def get_parameters(self):
        return self.model.get_weights()

    async def fit(self, parameters):
        self.model.set_weights(parameters)
        self.local_data = load_local_data()  # Load local data
        synthetic_data = generate_synthetic_data(self.gan_generator)
        combined_data = np.concatenate((self.local_data, synthetic_data))
        local_model = train_local_model(combined_data)
        return local_model.get_weights()

# Define a virtual federated server
class GANFedAvgServer(FederatedServer):
    def __init__(self, model, gan_generator):
        super().__init__()
        self.model = model
        self.gan_generator = gan_generator

    async def on_start(self):
        logger.log("Server has started!")

# Initialize the global model (modify as needed)
global_model = define_generator(10, 9)

# Load and preprocess the data (replace with your own data loading and preprocessing code)
def load_local_data():
    data = pd.read_csv('G:\Rakuten\diabetes.csv')
    # Preprocess the data as needed (e.g., normalization, one-hot encoding)
    return X, y

# Generate synthetic data using the GAN generator
def generate_synthetic_data(generator):
    latent_points = np.random.randn(750, 10)  # Modify the number of samples as needed
    X_synthetic = generator.predict(latent_points)
    return X_synthetic

# Train a local machine learning model (e.g., RandomForest) on the combined real and synthetic data
def train_local_model(data):
    features = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age']
    label = ['Outcome']
    X_train, X_test, y_train, y_test = train_test_split(data[features], data[label], test_size=0.3, random_state=42)
    clf = RandomForestClassifier(n_estimators=100)
    clf.fit(X_train, y_train)
    return clf

# Main function to run the Federated Learning process
async def main():
    num_clients = 10  # Number of clients
    num_rounds = 10  # Number of federated learning rounds

    # Create and initialize the GAN models
    latent_dim = 10
    discriminator = define_discriminator(9)
    generator = define_generator(latent_dim)
    gan_model = define_gan(generator, discriminator)

    # Create virtual federated clients
    clients = [GANFedAvgClient(model=global_model, gan_generator=generator) for _ in range(num_clients)]

    # Create virtual federated server
    server = GANFedAvgServer(model=global_model, gan_generator=generator)
    server.set_clients(clients)

    # Start federated server and run rounds
    await server.start()
    for round_num in range(num_rounds):
        logger.log(f"Round {round_num + 1}/{num_rounds}")
        await server.do_round()

    # Evaluation on the global model
    # Replace with your evaluation code
    test_data = load_test_data()
    accuracy = evaluate(global_model, test_data)
    logger.log("Global Model Accuracy:", accuracy)

if __name__ == "__main__":
    tf.config.experimental.set_visible_devices([], 'GPU')  # Disable GPU (if needed)
    await main()

ImportError: cannot import name 'FederatedServer' from 'flower' (C:\Users\Saubh\anaconda3\Lib\site-packages\flower\__init__.py)

from flower import FederatedServer, FederatedClient, NumPyDataset